In [1]:
import numpy as np
import cv2

def tilt_image(img,rotx, roty, rotz):
    # Set rotation angles and focal length
    rotx, roty, rotz = np.deg2rad(rotx),np.deg2rad(roty),np.deg2rad(rotz)
    f = 10

    # Get image dimensions
    h, w,_ = img.shape
    print(h,w)
    # Calculate sine and cosine of rotation angles
    cx, sx = np.cos(rotx), np.sin(rotx)
    cy, sy = np.cos(roty), np.sin(roty)
    cz, sz = np.cos(rotz), np.sin(rotz)

    # Compute rotation matrix
    roto = np.array([
        [cz * cy, cz * sy * sx - sz * cx, 0],
        [sz * cy, sz * sy * sx + cz * cx, 0],
        [-sy, cy * sx, 0]
    ])

    # Define the 4 corners of the input image
    pt = np.array([
        [-w / 2, -h / 2],
        [w / 2, -h / 2],
        [w / 2, h / 2],
        [-w / 2, h / 2]
    ])
    print(pt)
    # Transform corners with rotation and focal length
    ptt = np.zeros_like(pt)
    for i in range(4):
        pz = pt[i][0] * roto[2][0] + pt[i][1] * roto[2][1]
        ptt[i][0] = w / 2 + (pt[i][0] * roto[0][0] + pt[i][1] * roto[0][1]) * f * h / (f * h + pz)
        ptt[i][1] = h / 2 + (pt[i][0] * roto[1][0] + pt[i][1] * roto[1][1]) * f * h / (f * h + pz)

    # Define input and output points for perspective transformation
    in_pt = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
    out_pt = np.float32(ptt)

    # Compute perspective transformation matrix
    transform = cv2.getPerspectiveTransform(in_pt, out_pt)

    # Apply perspective transformation to input image
    img_in = img.copy()
    img = cv2.warpPerspective(img_in, transform, img_in.shape[:2])
    return img


In [2]:
def rotateImage(input, alpha, beta, gamma, dx, dy, dz, f):
    alpha = (alpha - 90.) * np.pi / 180.
    beta = (beta - 90.) * np.pi / 180.
    gamma = (gamma - 90.) * np.pi / 180.
    # get width and height for ease of use in matrices
    w = input.shape[1]
    h = input.shape[0]
    # Projection 2D -> 3D matrix
    A1 = np.array([[1, 0, -w/2],
    [0, 1, -h/2],
    [0, 0, 0],
    [0, 0, 1]], dtype=np.float64)
    # Rotation matrices around the X, Y, and Z axis
    RX = np.array([[1, 0, 0, 0],
    [0, np.cos(alpha), -np.sin(alpha), 0],
    [0, np.sin(alpha), np.cos(alpha), 0],
    [0, 0, 0, 1]], dtype=np.float64)
    RY = np.array([[np.cos(beta), 0, -np.sin(beta), 0],
    [0, 1, 0, 0],
    [np.sin(beta), 0, np.cos(beta), 0],
    [0, 0, 0, 1]], dtype=np.float64)
    RZ = np.array([[np.cos(gamma), -np.sin(gamma), 0, 0],
    [np.sin(gamma), np.cos(gamma), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]], dtype=np.float64)
    # Composed rotation matrix with (RX, RY, RZ)
    R = np.matmul(np.matmul(RX, RY), RZ)
    # Translation matrix
    T = np.array([[1, 0, 0, dx],
    [0, 1, 0, dy],
    [0, 0, 1, dz],
    [0, 0, 0, 1]], dtype=np.float64)
    # 3D -> 2D matrix
    A2 = np.array([[f, 0, w/2, 0],
    [0, f, h/2, 0],
    [0, 0, 1, 0]], dtype=np.float64)
    # Final transformation matrix
    trans = np.matmul(A2, np.matmul(T, np.matmul(R, A1)))
    # Apply matrix transformation
    output = cv2.warpPerspective(input, trans, (w,h//2), flags=cv2.INTER_LANCZOS4)
    return output,trans

In [3]:
def unrotate(image,trans):
    w = image.shape[1]
    h = image.shape[0]
    inv = np.linalg.inv(trans)
    # Apply matrix transformation
    print(inv)
    output = cv2.warpPerspective(image, inv, (w,h//2), flags=cv2.INTER_LANCZOS4)
    return output,inv

In [7]:
# Load the image
image = cv2.imread('image.jpg')
import time
# Tilt the image by 45 degrees
# tilted_image = tilt_image(image,30,0,00)
# print(tilted_image.shape, image.shape)
angle = 10
start = time.time()
tilted_image,trans = rotateImage( image,90-angle, 90, 90, 0, 0,200,200)
print(image.shape,tilted_image.shape)
print(time.time()-start)
start = time.time()
untilted_image,trans = unrotate(image,trans) #rotateImage( image,90+angle, 90, 90, 0, 0,200,200)
print(time.time()-start)
print(untilted_image)
# Display the tilted image
# cv2.imshow('Image', image)
cv2.imshow('Tilted Image', tilted_image)
cv2.imshow('unTilted Image', untilted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


(1280, 720, 3) (640, 720, 3)
0.009972095489501953
0.009971380233764648
[[[ 62 123 103]
  [ 63 124 104]
  [ 63 124 104]
  ...
  [ 90  90  70]
  [ 90  90  70]
  [ 91  90  70]]

 [[ 62 123 103]
  [ 62 123 103]
  [ 63 124 104]
  ...
  [ 90  89  69]
  [ 90  89  69]
  [ 90  89  69]]

 [[ 62 123 103]
  [ 63 124 104]
  [ 63 124 104]
  ...
  [ 90  89  69]
  [ 90  89  69]
  [ 90  89  69]]

 ...

 [[  0 178 211]
  [  0 173 206]
  [  0 174 205]
  ...
  [  2 108 127]
  [  0  98 115]
  [  1  92 110]]

 [[  4 197 231]
  [  1 171 199]
  [  0 178 207]
  ...
  [ 65 150 156]
  [ 28 102 107]
  [  0  76  82]]

 [[  3 201 235]
  [  1 170 198]
  [  0 182 211]
  ...
  [ 53 119 120]
  [ 11  61  61]
  [  0  57  55]]]


In [9]:
#http://jepsonsblog.blogspot.com/2012/11/rotation-in-3d-using-opencvs.html